**This notebook includes implementations of the following algorithms:**


*   Value Iteration Algorithm
*   Policy Iteration Algorithm

The notebook also includes a test gridworld game where the two algorithms are implemented to extract the optimal policy using value iteration and policy iteration.

Below is the commented implementation where each section is in a seperate notebook.


In [39]:
import numpy as np

**Grid world class representing the dynamics of the grid (enviroment) including the following:**


*   Grid size
*   Immediate rewards
*   Possible actions
*   States of the game which are indeed the cells in the grid





In [40]:
class GridWorld:

    def __init__(self, size):
        self.size = size
        self.actions = ['UP', 'DOWN', 'RIGHT', 'LEFT']
        self.discount = 0.99
        self.rewards = np.array([[0, -1, 10],
                                  [-1, -1, -1],
                                  [-1, -1, -1]])
        self.transitions = {'UP': [0.8, 0.1, 0.1, 0],
                            'DOWN': [0.1, 0.8, 0, 0.1],
                            'RIGHT': [0.1, 0, 0.8, 0.1],
                            'LEFT': [0.1, 0.1, 0, 0.8]}

    def next_position(self, i, j, action):
        if action == 'UP':
            return max(i - 1, 0), j
        elif action == 'DOWN':
            return min(i + 1, self.size - 1), j
        elif action == 'RIGHT':
            return i, min(j + 1, self.size - 1)
        elif action == 'LEFT':
            return i, max(j - 1, 0)

**In this section, the class agent algorithms is implemented including the following:**


*   Value Iteration: finds the optimal value function for each state
*   Policy Extraction: finds the optimal policy based on the output of the value iteration.
*   Policy Iteration



In [41]:
class Agent:

    def __init__(self, world: GridWorld):
        self.world = world

    def value_iteration(self):
        epsilon = 0.01
        while True:
            change = 0
            for i in range(self.world.size):
                for j in range(self.world.size):
                    if self.world.rewards[i][j] in [self.world.rewards[0][0], 10]:
                        continue
                    old_value = self.world.value_function[i][j]
                    new_value = float('-inf')
                    for action in self.world.actions:
                        action_probs = self.world.transitions[action]
                        action_value = 0
                        for k, prob in enumerate(action_probs):
                            new_i, new_j = self.world.next_position(i, j, self.world.actions[k])
                            if self.world.rewards[new_i][new_j] in [-3, 10]:
                                action_value += prob * self.world.rewards[new_i][new_j]
                            else:
                                action_value += prob * (self.world.rewards[new_i][new_j] +
                                                         self.world.discount * self.world.value_function[new_i][new_j])
                        if action_value > new_value:
                            new_value = action_value
                    self.world.value_function[i][j] = new_value
                    change = max(change, abs(old_value - self.world.value_function[i][j]))
            if change < epsilon:
                break

    def optimal_policy(self, reward):
        self.world.rewards[0][0] = reward
        self.world.value_function = np.zeros((self.world.size, self.world.size))
        self.value_iteration()
        policy = np.zeros((self.world.size, self.world.size), dtype=str)
        for i in range(self.world.size):
            for j in range(self.world.size):
                if self.world.rewards[i][j] in [reward, 10]:
                    continue
                best_action = None
                best_value = float('-inf')
                for action in self.world.actions:
                    action_probs = self.world.transitions[action]
                    action_value = 0
                    for k, prob in enumerate(action_probs):
                        new_i, new_j = self.world.next_position(i, j, self.world.actions[k])
                        if self.world.rewards[new_i][new_j] in [-3, 10]:
                            action_value += prob * self.world.rewards[new_i][new_j]
                        else:
                            action_value += prob * (self.world.rewards[new_i][new_j] +
                                                     self.world.discount * self.world.value_function[new_i][new_j])
                    if action_value > best_value:
                        best_value = action_value
                        best_action = action
                policy[i][j] = best_action
        policy[0][0] = 'T'
        policy[0][2] = 'T'
        return self.world.value_function, policy

**In this section**, The test cases are implemented using different values for the variable reward r and a discount factor = 0.99 (The future is highly accounted in the calculation)

In [42]:
grid = GridWorld(size=3)
agent = Agent(grid)
rewards = [100, 3, 0, -3]
for r in rewards:
    value_function, optimal_policy = agent.optimal_policy(r)
    print(f"r = {r}:")
    print("-------")
    print("Value Function (using Value Iteration):")
    print("---------------------------------------")
    print(value_function)
    print("\nPolicy (Optimal using Policy Iteration):")
    print("----------------------------------------")
    print(optimal_policy)
    print("===============================================================")

r = 100:
-------
Value Function (using Value Iteration):
---------------------------------------
[[ 0.         99.16163177  0.        ]
 [98.8595135  96.41608506 85.45794401]
 [96.11381005 93.96967881 90.88072671]]

Policy (Optimal using Policy Iteration):
----------------------------------------
[['T' 'L' 'T']
 ['U' 'L' 'L']
 ['U' 'L' 'L']]
r = 3:
-------
Value Function (using Value Iteration):
---------------------------------------
[[0.         9.10099888 0.        ]
 [6.46104359 8.10949731 9.55692641]
 [5.67926929 6.91829271 8.19072475]]

Policy (Optimal using Policy Iteration):
----------------------------------------
[['T' 'R' 'T']
 ['R' 'R' 'U']
 ['R' 'U' 'U']]
r = 0:
-------
Value Function (using Value Iteration):
---------------------------------------
[[0.         8.76803551 0.        ]
 [6.06459772 8.0372858  9.55692641]
 [5.57989863 6.85481643 8.19072475]]

Policy (Optimal using Policy Iteration):
----------------------------------------
[['T' 'R' 'T']
 ['R' 'R' 'U']
 ['R' 